# Pyclient for ESGF Data download

In [ ]:
# https://github.com/cehbrecht/jupyterlab-notebooks/blob/master/esgf-examples/esgf-pydap.ipynb

In [ ]:
# https://esgf-pyclient.readthedocs.io/en/latest/notebooks/examples/download.html

In [ ]:
# https://towardsdev.com/tricks-and-best-practices-from-kaggle-794a5914480f

## Import packages

In [1]:
from pyesgf.search import SearchConnection
import os
import pandas as pd
import requests
from tqdm import tqdm
import xarray as xr
import netCDF4

## LOGIN CREDENTIALS

In [3]:
from pyesgf.logon import LogonManager
lm = LogonManager()
lm.logoff()
lm.is_logged_on()

In [ ]:
lm.logon(hostname="esgf-data.dkrz.de", bootstrap=True, interactive=True)
#Enter myproxy username:OLUWAROTIMI
#mypassword='Odou1@@@@@@@'
lm.is_logged_on()

In [ ]:
#Secure good session and next authentification path
openid='https://esgf-data.dkrz.de/esgf-idp/openid/OLUWAROTIMI'
password='Odou1@@@@@@@'
from pydap.client import open_url
from pydap.cas.esgf import setup_session
dataset_url ='http://esg-dn1.nsc.liu.se/thredds/dodsC/esg_dataroot1/cordexdata/cordex/output/AFR-44/SMHI/CNRM-CERFACS-CNRM-CM5/historical/r1i1p1/SMHI-RCA4/v1/day/tasmax/v20130927/tasmax_AFR-44_CNRM-CERFACS-CNRM-CM5_historical_r1i1p1_SMHI-RCA4_v1_day_19510101-19551231.nc'
session = setup_session(openid, password, check_url=dataset_url, verify=True)
dataset = open_url(dataset_url, session=session)

## SearchConnection: instances represent a connection to an ESGF Search web service. This stores the service URL and also service-level parameters like distrib and shards

In [4]:
CORDEX_MODEL=['CNRM-CERFACS-CNRM-CM5','CSIRO-QCCCE-CSIRO-Mk3-6-0','IPSL-IPSL-CM5A-MR','MIROC-MIROC5','MPI-M-MPI-ESM-LR',
              'NCC-NorESM1-M','NOAA-GFDL-GFDL-ESM2M']

In [ ]:
from  pyesgf.search  import  SearchConnection
#conn = SearchConnection('http://esgf-data.dkrz.de/esg-search/', distrib=True)
conn = SearchConnection('https://esgf-node.llnl.gov/esg-search', distrib=True)

In [6]:
ctx = conn.new_context(
project='CORDEX',
time_frequency='day',
ensemble='r1i1p1',
experiment='historical,rcp45,rcp85',
domain='AFR-44',
driving_model=CORDEX_MODEL,
#driving_model='CNRM-CERFACS-CNRM-CM5',
rcm_name='RCA4',
variable='tas,tasmax,tasmin',
latest=True,
#data_node='esgf-data.dkrz.de'
)
ctx.hit_count

63

## Execute the search

In [14]:
# for each_hit in range(0,ctx.hit_count):
#     print(str(each_hit)+'yy')

In [ ]:
from  pyesgf.logon  import  LogonManager
lm = LogonManager()
lm.logoff()
lm.is_logged_on()

In [ ]:
myproxy_host = 'esgf-data.dkrz.de'
lm.logon(username='OLUWAROTIMI', password='Odou1@@@@@@@', hostname=myproxy_host)
lm.is_logged_on()

In [16]:
import  tempfile
import  os, subprocess

In [19]:
ds = ctx.search()
for each_hit in range(0,ctx.hit_count):
    
    print('=========Hit Process '+str(each_hit)+'/'+str(ctx.hit_count)+' Going on==========')
    
    fc=ds[each_hit].file_context()
    wget_script_content = fc.get_download_script() #get script contents
    
    files=ds[each_hit].file_context().search()
    files_opendap_url=sorted([i.opendap_url for i in files])
    files_label=sorted([i.filename for i in files])
    
    outfilename='download_'+files_label[0][0:-20]+files_label[0][-20:-12]+'-'+files_label[len(files_label)-1][-11:-3]
    script_path = tempfile.mkstemp(suffix='.sh', prefix=outfilename)[1]
    
    with  open(script_path, "w") as  writer:
        writer.write(wget_script_content)
        writer.close()
    
    os.chmod(script_path, 0o750)
    
    print ('File created : '+str(script_path)+'from hit '+str(each_hit))